## <font color='#32CD32'> Preprocessing Data </font>

### Impor Library

In [2]:
import pandas as pd
import numpy as np

### Impor Data

In [3]:
data = pd.read_csv('DataCoSupply.csv',header= 0,encoding='unicode_escape')
data.head(5)

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,Customer City,...,Order Zipcode,Product Card Id,Product Category Id,Product Description,Product Image,Product Name,Product Price,Product Status,shipping date (DateOrders),Shipping Mode
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,2/3/2018 22:56,Standard Class
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/18/2018 12:27,Standard Class
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,San Jose,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/17/2018 12:06,Standard Class
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,73,Sporting Goods,Los Angeles,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/16/2018 11:45,Standard Class
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/15/2018 11:24,Standard Class


In [4]:
data.shape

(180519, 53)

### Cek Missing Value

In [5]:
def features_with_missing_values(data):
    x = data.isnull().sum().sum()/(data.shape[0]*data.shape[1])*100
    print('Percentage of Total Missing Values is ' ,round(x,2) ,'%')
    print('Missing Value Estimation :')
    for i in data.columns:
        if data[i].isna().sum()>0:
            print('The Feature ',i,' has '+ str(data[i].isna().sum()) + ' missing values')
            
features_with_missing_values(data)

Percentage of Total Missing Values is  3.51 %
Missing Value Estimation :
The Feature  Customer Lname  has 8 missing values
The Feature  Customer Zipcode  has 3 missing values
The Feature  Order Zipcode  has 155679 missing values
The Feature  Product Description  has 180519 missing values


### Memilih Fitur Untuk Pemodelan

In [6]:
# Mengubah variabel waktu dan tanggal ke dalam format pandas
data['order_date'] = pd.to_datetime(data['order date (DateOrders)'])
data['shipping_date']=pd.to_datetime(data['shipping date (DateOrders)'])

data['order_year'] = pd.DatetimeIndex(data['order_date']).year
data['order_month'] = pd.DatetimeIndex(data['order_date']).month
data['order_day'] = pd.DatetimeIndex(data['order_date']).day
data['shipping_year'] = pd.DatetimeIndex(data['shipping_date']).year
data['shipping_month'] = pd.DatetimeIndex(data['shipping_date']).month
data['shipping_day'] = pd.DatetimeIndex(data['shipping_date']).day

In [7]:
fitur_pilihan = ['Type','Days for shipping (real)','Days for shipment (scheduled)','Late_delivery_risk','Benefit per order',
                        'Sales per customer','Latitude','Longitude','Shipping Mode','Order Status','Order Region',
                        'Order Country','Order City','Market','Delivery Status','order_day','order_month','order_year',
                        'shipping_day','shipping_month','shipping_year']

new_data = data[fitur_pilihan]
model_data = new_data.copy()

In [9]:
from sklearn.preprocessing import LabelEncoder

# Fitur yang ingin saya transformasi
categorical_features = ['Type', 'Shipping Mode', 'Order Status', 'Order Region', 'Order Country', 'Order City', 'Market', 'Delivery Status']

def apply_label_encoders(df, features):
    # Dictionary untuk menyimpan encoder yang sesuai dengan setiap kolom
    label_encoders = {}
    
    for col in features:
        # Buat LabelEncoder untuk setiap kolom
        le = LabelEncoder()
        
        # Fit dan transform kolom
        df[col] = le.fit_transform(df[col])
        
        # Simpan encoder ke dalam dictionary untuk penggunaan di masa depan
        label_encoders[col] = le
    
    return df, label_encoders

# Terapkan fungsi ke DataFrame
model_data, encoders = apply_label_encoders(model_data, categorical_features)
model_data

,Type,Days for shipping (real),Days for shipment (scheduled),Late_delivery_risk,Benefit per order,Sales per customer,Latitude,Longitude,Shipping Mode,Order Status,...,Order Country,Order City,Market,Delivery Status,order_day,order_month,order_year,shipping_day,shipping_month,shipping_year
0,1,3,4,0,91.250000,314.640015,18.251453,-66.037056,3,2,...,70,331,3,0,31,1,2018,3,2,2018
1,3,5,4,1,-249.089996,311.359985,18.279451,-66.037064,3,5,...,69,391,3,1,13,1,2018,18,1,2018
2,0,4,4,0,-247.779999,309.720001,37.292233,-121.881279,3,1,...,69,391,3,3,13,1,2018,17,1,2018
3,1,3,4,0,22.860001,304.809998,34.125946,-118.291016,3,2,...,8,3226,3,0,13,1,2018,16,1,2018
4,2,2,4,0,134.210007,298.250000,18.253769,-66.037048,3,6,...,8,3226,3,0,13,1,2018,15,1,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180514,0,4,4,0,40.000000,399.980011,40.640930,-73.942711,3,1,...,31,2922,3,3,16,1,2016,20,1,2016
180515,1,3,2,1,-613.770019,395.980011,35.362545,-119.018700,2,2,...,77,1362,3,1,16,1,2016,19,1,2016
180516,3,5,4,1,141.110001,391.980011,41.629959,-72.967155,3,5,...,8,25,3,1,15,1,2016,20,1,2016
180517,2,3,4,0,186.229996,387.980011,18.213350,-66.370575,3,6,...,8,25,3,0,15,1,2016,18,1,2016


In [12]:
# model_data = pd.get_dummies(model_data)
# model_data.shape

### Memisahkan data menjadi X dan y

In [13]:
X = model_data.drop(['Days for shipping (real)','Days for shipment (scheduled)'],axis=1)
y = model_data[['Days for shipping (real)','Days for shipment (scheduled)']].values
X.shape,y.shape

((180519, 19), (180519, 2))

### Memisahkan data menjadi data train dan test


In [14]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=42)
X_train1 = X_train.drop(['Late_delivery_risk'], axis=1).values
X_test1 = X_test.drop(['Late_delivery_risk'], axis=1).values
X_train1.shape, X_test1.shape

((144415, 18), (36104, 18))

## <font color='#32CD32'> Decision Tree </font>

In [145]:
from sklearn.tree import DecisionTreeRegressor

model_dt = DecisionTreeRegressor()
model_dt.fit(X_train1,y_train)

DecisionTreeRegressor()

In [146]:
y_pred = model_dt.predict(X_test1)
Prediction = pd.DataFrame(y_pred)
prediction = Prediction.rename(columns={0:'Fastest_shipment',1:'Avg_shipment'})
prediction['risk'] = np.where(prediction['Avg_shipment'] >= prediction['Fastest_shipment'],0,1)
prediction.head()

,Fastest_shipment,Avg_shipment,risk
0,5.0,4.0,1
1,2.0,1.0,1
2,2.0,4.0,0
3,5.0,4.0,1
4,2.0,4.0,0


In [147]:
from sklearn.metrics import classification_report

l = prediction['risk']
m = X_test['Late_delivery_risk']
l.shape,m.shape

print(classification_report(m, l))

              precision    recall  f1-score   support

           0       1.00      0.94      0.97     16307
           1       0.96      1.00      0.98     19797

    accuracy                           0.98     36104
   macro avg       0.98      0.97      0.97     36104
weighted avg       0.98      0.98      0.97     36104



## <font color='#32CD32'> Random Forest </font>

In [15]:
from sklearn.ensemble import RandomForestRegressor

model_rf = RandomForestRegressor()
model_rf.fit(X_train1,y_train)

RandomForestRegressor()

### Data Train

In [16]:
y_pred = model_rf.predict(X_train1)
Prediction = pd.DataFrame(y_pred)
prediction = Prediction.rename(columns={0:'Fastest_shipment',1:'Avg_shipment'})
prediction['risk'] = np.where(prediction['Avg_shipment'] >= prediction['Fastest_shipment'],0,1)
prediction.head()

,Fastest_shipment,Avg_shipment,risk
0,2.0,1.0,1
1,2.0,2.0,0
2,2.0,1.0,1
3,2.0,1.0,1
4,2.0,1.0,1


In [17]:
from sklearn.metrics import classification_report

l = prediction['risk']
m = X_train['Late_delivery_risk']
l.shape,m.shape

print(classification_report(m, l))

              precision    recall  f1-score   support

           0       1.00      0.93      0.97     65235
           1       0.95      1.00      0.97     79180

    accuracy                           0.97    144415
   macro avg       0.97      0.97      0.97    144415
weighted avg       0.97      0.97      0.97    144415



### Data Test

In [18]:
y_pred = model_rf.predict(X_test1)
Prediction = pd.DataFrame(y_pred)
prediction = Prediction.rename(columns={0:'Fastest_shipment',1:'Avg_shipment'})
prediction['risk'] = np.where(prediction['Avg_shipment'] >= prediction['Fastest_shipment'],0,1)
prediction.head()

,Fastest_shipment,Avg_shipment,risk
0,5.04,4.0,1
1,2.00,1.0,1
2,2.20,4.0,0
3,5.03,4.0,1
4,2.07,4.0,0


In [19]:
from sklearn.metrics import classification_report

l = prediction['risk']
m = X_test['Late_delivery_risk']
l.shape,m.shape

print(classification_report(m, l))

              precision    recall  f1-score   support

           0       1.00      0.93      0.96     16307
           1       0.95      1.00      0.97     19797

    accuracy                           0.97     36104
   macro avg       0.97      0.97      0.97     36104
weighted avg       0.97      0.97      0.97     36104



## <font color='#32CD32'> Gradien Boosting Regression </font>

In [153]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor

model_gb = MultiOutputRegressor(GradientBoostingRegressor())
model_gb.fit(X_train1, y_train)

MultiOutputRegressor(estimator=GradientBoostingRegressor())

In [154]:
y_pred = model_gb.predict(X_test1)
Prediction = pd.DataFrame(y_pred)
prediction = Prediction.rename(columns={0:'Fastest_shipment',1:'Avg_shipment'})
prediction['risk'] = np.where(prediction['Avg_shipment'] >= prediction['Fastest_shipment'],0,1)
prediction.head()

,Fastest_shipment,Avg_shipment,risk
0,5.380637,3.999972,1
1,2.005525,1.000051,1
2,2.395259,3.999972,0
3,5.573523,3.999972,1
4,2.490374,3.999972,0


In [155]:
from sklearn.metrics import classification_report

l = prediction['risk']
m = X_test['Late_delivery_risk']
l.shape,m.shape

print(classification_report(m, l))

              precision    recall  f1-score   support

           0       1.00      0.78      0.88     16307
           1       0.85      1.00      0.92     19797

    accuracy                           0.90     36104
   macro avg       0.92      0.89      0.90     36104
weighted avg       0.92      0.90      0.90     36104



## <font color='#32CD32'> XGBoost </font>

In [156]:
import xgboost as xgb
model_xgb = xgb.XGBRegressor(n_estimators=100, 
                              learning_rate=0.1, 
                              random_state=42)

model_xgb.fit(X_train1, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [157]:
y_pred = model_xgb.predict(X_test1)
Prediction = pd.DataFrame(y_pred)
prediction = Prediction.rename(columns={0:'Fastest_shipment',1:'Avg_shipment'})
prediction['risk'] = np.where(prediction['Avg_shipment'] >= prediction['Fastest_shipment'],0,1)
prediction.head()

,Fastest_shipment,Avg_shipment,risk
0,5.179741,3.999979,1
1,2.000952,1.000059,1
2,2.142590,3.999979,0
3,5.226353,3.999979,1
4,2.166739,3.999979,0


In [158]:
from sklearn.metrics import classification_report

l = prediction['risk']
m = X_test['Late_delivery_risk']
l.shape,m.shape

print(classification_report(m, l))

              precision    recall  f1-score   support

           0       1.00      0.82      0.90     16307
           1       0.87      1.00      0.93     19797

    accuracy                           0.92     36104
   macro avg       0.94      0.91      0.92     36104
weighted avg       0.93      0.92      0.92     36104



## <font color='#32CD32'> ANN </font>

In [159]:
import tensorflow as tf

model_ann = tf.keras.models.Sequential([
    tf.keras.Input(shape=(X_train1.shape[1],)),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(22, activation='relu'),
    tf.keras.layers.Dense(2, activation='linear')
])

model_ann.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
model_ann.summary()

2024-12-14 02:55:57.088564: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-14 02:55:57.333469: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-14 02:55:57.406445: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734119757.612407   37605 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734119757.658744   37605 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-14 02:55:58.036569: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 24)             │           456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 22)             │           550 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │            46 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,052 (4.11 KB)

 Trainable params: 1,052 (4.11 KB)

 Non-trainable params: 0 (0.00 B)

In [160]:
Mymodel = model_ann.fit(X_train1, y_train, epochs=100,  validation_split=0.3, verbose=1)

Epoch 1/100
5055/5055 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - accuracy: 0.5196 - loss: 8813.8691 - val_accuracy: 0.7529 - val_loss: 4.9267
Epoch 2/100
5055/5055 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - accuracy: 0.6046 - loss: 4.1628 - val_accuracy: 0.7083 - val_loss: 2.1609
Epoch 3/100
5055/5055 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - accuracy: 0.6156 - loss: 3.0723 - val_accuracy: 0.5393 - val_loss: 2.5990
Epoch 4/100
5055/5055 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - accuracy: 0.6151 - loss: 2.8435 - val_accuracy: 0.7573 - val_loss: 3.8084
Epoch 5/100
5055/5055 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - accuracy: 0.6125 - loss: 2.6303 - val_accuracy: 0.8048 - val_loss: 2.6130
Epoch 6/100
5055/5055 ━━━━━━━━━━━━━━━━━━━━ 15s 3ms/step - accuracy: 0.6287 - loss: 2.2901 - val_accuracy: 0.7574 - val_loss: 5.0380
Epoch 7/100
5055/5055 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - accuracy: 0.6299 - loss: 2.3168 - val_accuracy: 0.5115 - val_loss: 2.2728
Epoch 8/100
5055/5055 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - accuracy: 0.652

In [161]:
y_pred = model_ann.predict(X_test1)
Prediction = pd.DataFrame(y_pred)
prediction = Prediction.rename(columns={0:'Fastest_shipment',1:'Avg_shipment'})
prediction['risk'] = np.where(prediction['Avg_shipment'] >= prediction['Fastest_shipment'],0,1)
prediction.head()

1129/1129 ━━━━━━━━━━━━━━━━━━━━ 1s 682us/step


,Fastest_shipment,Avg_shipment,risk
0,5.283577,4.193332,1
1,1.984897,0.978877,1
2,3.018461,4.122717,0
3,5.309527,4.233542,1
4,3.314292,4.149123,0


In [162]:
from sklearn.metrics import classification_report

l = prediction['risk']
m = X_test['Late_delivery_risk']
l.shape,m.shape

print(classification_report(m, l))

              precision    recall  f1-score   support

           0       1.00      0.51      0.68     16307
           1       0.71      1.00      0.83     19797

    accuracy                           0.78     36104
   macro avg       0.86      0.76      0.75     36104
weighted avg       0.84      0.78      0.76     36104



## <font color='#32CD32'> Model yang digunakan </font>

Saya akan menggunakan model random forest, karena decision tree dikhawatirkan mengalami overfitting.

In [20]:
model_rf.predict([[1, 91.250000, 314.640015 , 18.251453, -66.037056, 3, 2, 15, 70, 331, 3, 0, 31, 1, 2018, 3,2, 2018]])

array([[2.7, 4. ]])

In [21]:
# save model
import joblib
# Simpan model ke file
joblib.dump(model_rf, 'model_rf.pkl')
joblib.dump(encoders, 'label_encoder.pkl')

['label_encoder.pkl']

In [22]:
new_data

,Type,Days for shipping (real),Days for shipment (scheduled),Late_delivery_risk,Benefit per order,Sales per customer,Latitude,Longitude,Shipping Mode,Order Status,...,Order Country,Order City,Market,Delivery Status,order_day,order_month,order_year,shipping_day,shipping_month,shipping_year
0,DEBIT,3,4,0,91.250000,314.640015,18.251453,-66.037056,Standard Class,COMPLETE,...,Indonesia,Bekasi,Pacific Asia,Advance shipping,31,1,2018,3,2,2018
1,TRANSFER,5,4,1,-249.089996,311.359985,18.279451,-66.037064,Standard Class,PENDING,...,India,Bikaner,Pacific Asia,Late delivery,13,1,2018,18,1,2018
2,CASH,4,4,0,-247.779999,309.720001,37.292233,-121.881279,Standard Class,CLOSED,...,India,Bikaner,Pacific Asia,Shipping on time,13,1,2018,17,1,2018
3,DEBIT,3,4,0,22.860001,304.809998,34.125946,-118.291016,Standard Class,COMPLETE,...,Australia,Townsville,Pacific Asia,Advance shipping,13,1,2018,16,1,2018
4,PAYMENT,2,4,0,134.210007,298.250000,18.253769,-66.037048,Standard Class,PENDING_PAYMENT,...,Australia,Townsville,Pacific Asia,Advance shipping,13,1,2018,15,1,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180514,CASH,4,4,0,40.000000,399.980011,40.640930,-73.942711,Standard Class,CLOSED,...,China,Shanghái,Pacific Asia,Shipping on time,16,1,2016,20,1,2016
180515,DEBIT,3,2,1,-613.770019,395.980011,35.362545,-119.018700,Second Class,COMPLETE,...,Japón,Hirakata,Pacific Asia,Late delivery,16,1,2016,19,1,2016
180516,TRANSFER,5,4,1,141.110001,391.980011,41.629959,-72.967155,Standard Class,PENDING,...,Australia,Adelaide,Pacific Asia,Late delivery,15,1,2016,20,1,2016
180517,PAYMENT,3,4,0,186.229996,387.980011,18.213350,-66.370575,Standard Class,PENDING_PAYMENT,...,Australia,Adelaide,Pacific Asia,Advance shipping,15,1,2016,18,1,2016


In [34]:
new_data['Delivery Status'].unique()

array(['Advance shipping', 'Late delivery', 'Shipping on time',
       'Shipping canceled'], dtype=object)

In [185]:
new_data['Shipping Mode'].unique()

array(['Standard Class', 'First Class', 'Second Class', 'Same Day'],
      dtype=object)

In [27]:
# Muat model dari file
ayam = joblib.load('model_rf.pkl')
encoder = joblib.load('label_encoder.pkl')

In [28]:
a = encoder['Order Status'].transform(['COMPLETE'])
print(a[0])

2


In [30]:
ayam.predict([[1, 91.250000, 314.640015 , 18.251453, -66.037056, 3, 2, 15, 70, 331, 3, 0, 31, 1, 2018, 3,2, 2018]])

array([[2.7, 4. ]])

In [170]:
# 1 91.250000 314.640015 18.251453 -66.037056 3 2 15 70 331 3 0 31 1 2018 3 2 2018

In [14]:
['COMPLETE', 'PENDING', 'CLOSED', 'PENDING_PAYMENT', 'CANCELED', 'PROCESSING', 'SUSPECTED_FRAUD', 'ON_HOLD', 'PAYMENT_REVIEW']

[2, 5, 1, 6, 0, 7, 8, 3, 4]

[2, 5, 1, 6, 0, 7, 8, 3, 4]